In [98]:
#import libraries
import requests
import pandas as pd
import time
import psycopg2 as ps
from dotenv import load_dotenv
import os

In [99]:
load_dotenv()

True

In [100]:
#Keys
API_KEY = os.getenv('API_KEY')
CHANNEL_ID = "UCCT_-OGW5IiUuuHwmuyUPYQ"

In [101]:
def get_video_details(video_id):
    
        # Collecting view, like, dislike, comment counts
        url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + "&part=statistics&key=" + API_KEY
        response_video_stats = requests.get(url_video_stats).json()

        statistics = response_video_stats['items'][0]['statistics']

        # Check if 'dislikeCount' exists before accessing it
        if 'dislikeCount' in statistics:
            dislike_count = statistics['dislikeCount']
        else:
            dislike_count = 0  # Set default value if dislike count is not available

        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']

        return view_count, like_count, dislike_count, comment_count

In [102]:
def get_videos(df,api_key=API_KEY,channel_id=CHANNEL_ID):
    # Make API call
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key="+api_key+"&channelId="+channel_id+"&part=snippet,id&order=date&maxResults=10000"+pageToken
    response = requests.get(url).json()
    time.sleep(1)

    # Create an empty list to store rows
    rows_to_concat = []

    for video in response['items']:
        if video['id']['kind'] == "youtube#video":
            video_id = video['id']['videoId']
            video_title = video['snippet']['title'].replace("&", "")
            upload_date = str(video['snippet']['publishedAt']).split("T")[0]

            view_count, like_count, dislike_count, comment_count = get_video_details(video_id)

            # Create a dictionary for the row
            row_dict = {'video_id': video_id, 'video_title': video_title,
                        'upload_date': upload_date, 'view_count': view_count,
                        'like_count': like_count, 'dislike_count': dislike_count,
                        'comment_count': comment_count}

            # Append the dictionary to the list
            rows_to_concat.append(row_dict)

    # Convert the list of dictionaries to a DataFrame
    df_new_rows = pd.DataFrame(rows_to_concat)

    # Concatenate the new rows with the existing DataFrame
    df = pd.concat([df, df_new_rows], ignore_index=True)

    return df


In [103]:
#main

#build our dataframe
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","dislike_count","comment_count"])
df = get_videos(df)

KeyError: 'likeCount'

In [ ]:
df.head()

In [ ]:
# Assuming 'df' is your pandas DataFrame
df.to_csv(r"C:\Users\Levin\Projects\4OceanSA.csv", index=False)


In [ ]:
df = pd.read_csv('4OceanSA.csv')
df.head()

In [ ]:
#connect to db
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
            raise e
    else:
        print('Connected!')
    return conn

In [ ]:
host_name = os.getenv('host_name')
dbname = os.getenv('dbname')
port = os.getenv('port')
username = os.getenv('un')
password = os.getenv('password')
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

In [ ]:
#create table
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                     video_id VARCHAR(255) PRIMARY KEY,
                     video_title TEXT NOT NULL,
                     upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                     view_count INTEGER NOT NULL,
                     like_count INTEGER NOT NULL,
                     dislike_count INTEGER NOT NULL,
                     comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

In [ ]:
#check if video exists
def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,))

    return curr.fetchone() is not None

In [ ]:
#update row if video exists

def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [ ]:
def update_db(curr, df):
    # Initialize an empty list to store rows for new videos
    new_videos = []

    for video_id, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']):  # if video already exists then we will update
            update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['dislike_count'], row['comment_count'])
        else:  # The video doesn't exist so we will collect rows to append to the db table later
            # Convert the Series to a DataFrame row and set 'video_id' as a column
            new_row_df = row.to_frame().T
            new_row_df['video_id'] = row['video_id']
            new_videos.append(new_row_df)

    # If there are new videos, concatenate them into a single DataFrame
    if new_videos:
        tmp_df = pd.concat(new_videos, ignore_index=True)
    else:
        # If there are no new videos, create an empty DataFrame with the specified columns
        tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'dislike_count', 'comment_count'])

    return tmp_df


In [ ]:
#write insert command

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [ ]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['dislike_count'], row['comment_count'])


In [ ]:
curr = conn.cursor()

In [ ]:
create_table(curr)

In [ ]:
new_vid_df = update_db(curr, df)

In [ ]:
append_from_df_to_db(curr, new_vid_df)

In [ ]:
conn.commit()